# AMPLPY: Jupyter Notebook Integration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplpy/blob/master/notebooks/magics.ipynb)

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplpy/blob/master/notebooks/magics.ipynb)

Documentation: http://amplpy.readthedocs.io

GitHub Repository: https://github.com/ampl/amplpy

PyPI Repository: https://pypi.python.org/pypi/amplpy

Jupyter Notebooks: https://github.com/ampl/amplpy/tree/master/notebooks

### Setup

In [1]:
!pip install -q amplpy ampltools

### Google Colab & Kaggle interagration

In [2]:
import os, shutil
def install(modules=None, license_uuid=None, run_once=True):
    """Installs AMPL bundle or individual modules."""
    from ampltools import ampl_installer
    ampl_dir = os.path.abspath(os.path.join(os.curdir, 'ampl.linux-intel64'))
    demo_lic_backup = os.path.join(ampl_dir, 'ampl.lic_demo')
    if run_once and os.path.isfile(demo_lic_backup):
        print('Already installed. Skipping.')
    elif modules is not None:
        for module in modules:
            ampl_installer(f'https://portal.ampl.com/dl/modules/{module}-module.linux64.tgz',
                           ampl_dir, verbose=True)
        shutil.copy(os.path.join(ampl_dir, 'ampl.lic'), demo_lic_backup)
    else:
        ampl_installer('https://portal.ampl.com/dl/amplce/ampl.linux64.tgz',
                       ampl_dir, verbose=True)
        shutil.copy(os.path.join(ampl_dir, 'ampl.lic'), demo_lic_backup)
    if license_uuid is not None:
        print(f'Activating license {license_uuid}')
        ampl_installer(f'https://portal.ampl.com/download/license/{license_uuid}/ampl.lic',
                       ampl_dir, verbose=True)
    else:
        print('Activating demo license.')
        shutil.copy(demo_lic_backup, os.path.join(ampl_dir, 'ampl.lic'))
    return ampl_dir

In [3]:
RUNNING_IN_GOOGLE_COLAB = 'COLAB_GPU' in os.environ
RUNNING_IN_KAGGLE = os.path.abspath(os.curdir).startswith('/kaggle/')
RUNNING_IN_THE_CLOUD = RUNNING_IN_GOOGLE_COLAB or RUNNING_IN_KAGGLE

In [4]:
# If you have an AMPL Cloud License or an AMPL CE license, you can use it on Google Colab and similar platforms.
# Note: Your license UUID should never be shared. Please make sure you delete the license UUID
# and rerun this cell before sharing the notebook with anyone.
LICENSE_UUID = None
# You can install individual modules from https://portal.ampl.com/dl/modules/
MODULES = ['ampl', 'gurobi']
# Set to True in order to install AMPL only once 
RUN_ONCE = True
if RUNNING_IN_THE_CLOUD:
    ampl_dir = install(modules=MODULES, license_uuid=LICENSE_UUID, run_once=RUN_ONCE)
    os.environ['PATH'] += os.pathsep + ampl_dir

### Imports

In [5]:
from amplpy import AMPL, register_magics


### Instantiate an ampl object and register jupyter magics

In [6]:
ampl = AMPL()
ampl.eval('option version;')
# Store %%ampl cells in the list _ampl_cells
# Evaluate %%ampl_eval cells with ampl.eval()
register_magics(store_name='_ampl_cells', ampl_object=ampl)

option version 'AMPL Version 20220119 (Darwin-19.6.0, 64-bit)\
Demo license with maintenance expiring 20240131.\
Using license file "/Users/fdabrandao/bin/ampl.macos64/ampl.lic".\
';


### Use `%%ampl_eval` to pass the model to AMPL

In [7]:
%%ampl_eval
set SIZES;
param capacity >= 0;
param value {SIZES};
var Qty {SIZES} binary;
maximize TotVal: sum {s in SIZES} value[s] * Qty[s];
subject to Cap: sum {s in SIZES} s * Qty[s] <= capacity;

### Set data

In [8]:
ampl.set['SIZES'] = [5, 4, 6, 3]
ampl.param['value'] = [10, 40, 30, 50]
ampl.param['capacity'] = 10

### Use `%%ampl_eval` to display values

In [9]:
%%ampl_eval
display SIZES;
display value;
display capacity;

set SIZES := 5 4 6 3;

value [*] :=
3  50
4  40
5  10
6  30
;

capacity = 10



### Use amplpy to retrive values

In [10]:
print('SIZES:', ampl.set['SIZES'].getValues().toList())
print('value:', ampl.param['value'].getValues().toDict())
print('capacity:', ampl.param['capacity'].value())

SIZES: [5.0, 4.0, 6.0, 3.0]
value: {3.0: 50.0, 4.0: 40.0, 5.0: 10.0, 6.0: 30.0}
capacity: 10.0


### Use `%%ampl_eval` to solve the model

In [11]:
%%ampl_eval
option solver gurobi;
option gurobi_options 'outlev=1';
solve;

gurobi (Darwin x86_64), licchk(20131021), ASL(20211109)
Gurobi 9.5.0: outlev=1
Set parameter OutputFlag to value 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x1dc33b55
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [3e+00, 6e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 50.0000000
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 90 50 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
